In [8]:
%load_ext autoreload
%autoreload 2
import sys
import gc

from multiprocessing import cpu_count, Pool
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as data_utils
import matplotlib.pyplot as plt

from tqdm import tqdm

sys.path.append('../')
from wiki.utils import clock
from wiki import rnn, rnn_predict, newphet, val, submissions
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
base_dir = '../data/'
train_df = pd.read_csv(base_dir+'train_2.csv')
true = train_df.drop('Page', axis=1).values

In [6]:
median = np.nan_to_num(np.nanmedian(true[:,-49:], axis=1).reshape(-1,1).clip(0).round())

/home/ubuntu/miniconda3/envs/basev1/lib/python3.6/site-packages/numpy/lib/function_base.py:3858: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


In [7]:
broadcast = np.broadcast_to(median, (median.shape[0], 62))

Val score for the below: 41.04

In [9]:
submissions.write_submission(broadcast, '../data/submissions/median_stage_2.csv')

Get RNN preds

In [10]:
loss_func = nn.L1Loss()
model = rnn.RNN(loss_func=loss_func).cuda()

In [11]:
batch_size = 1024
pred_len = 62

In [12]:
base_dir = '../data/'
train_df = pd.read_csv(base_dir+'train_2.csv').fillna(0)

In [13]:
X = train_df.drop('Page', axis=1).values
X, scaler = rnn.scale_values(X)

In [14]:
testloader = data_utils.DataLoader(
    data_utils.TensorDataset(
        torch.from_numpy(X[:,pred_len:,:]).float(),
        torch.from_numpy(X[:,-pred_len:,:]).float()
    ),
    batch_size=batch_size, shuffle=False
)

In [15]:
save_best_path = base_dir+'rnn_stage2_FINAL_v2_lr2.mdl'
model.load_state_dict(torch.load(save_best_path))

In [16]:
outputs, targets, sequences = model.predict(testloader)

In [17]:
_, predictions = rnn_predict.combine_prediction_data(outputs, targets, sequences)

In [18]:
predictions = scaler.inverse_transform(predictions.T).T

In [19]:
predictions = predictions.round().clip(0)

In [24]:
predictions[:,-62:].shape

(145063, 62)

Val score for the below: 43.34

In [26]:
fp = '../data/submissions/rnn_v2_final.csv'
submissions.write_submission(predictions[:,-62:], fp)

Blend 1 (static)

Val score for the below: 40.55

In [30]:
fp = '../data/submissions/rnn_v2_median_static_blend_1.csv'
submissions.write_submission((1.5*median+predictions[:,-62:])/2.5, fp)

Blend 2 (variable)

In [31]:
blended_roll = np.stack(
    [(predictions[:, -62+x]*(2-x*0.05)+broadcast[:,-62+x]*(1+x*0.05))/3 for x in range(20)],
    axis=1
)
blended_roll = np.concatenate([blended_roll, (2*broadcast[:,-42:]+predictions[:,-42:])/3], axis=1)

In [32]:
blended_roll.shape

(145063, 62)

Val score for the below: 40.49

In [33]:
fp = '../data/submissions/rnn_v2_median_variable_blend_1.csv'
submissions.write_submission(blended_roll, fp)